# Playground for ASFPy module development

In [1]:
import os

from asfpy import asfpy

In [2]:
categories = [
    "Clinical",
    "Developmental",
    "Affective",
    "Social",
    "Cognitive"
]

In [3]:
applicants = [
    {
        "id": "APP001",
        "urm": False,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Clinical"
        },
    },
    {
        "id": "APP002",
        "urm": True,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Cognitive"
        },
    },
    {
        "id": "APP003",
        "urm": True,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Affective",
            "Cognitive"
        }
    },
    {
        "id": "APP004",
        "urm": True,
        "lim": True,
        "du": False,
        "is_flexible": False,        
        "categories": {
            "Clinical"
        }
    },
    {
        "id": "APP005",
        "urm": False,
        "lim": True,
        "du": False,
        "is_flexible": True,        
        "categories": {
            "Developmental"
        }
    }
]

In [4]:
editors = [
    {
        "id": "EDI001",
        "role": "faculty",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 3
    },
    {
        "id": "EDI002",
        "role": "faculty",
        "categories": {
            "Affective"
        },
        "capacity": 2
    },
    {
        "id": "EDI003",
        "role": "faculty",
        "categories": {
            "Cognitive",
            "Affective"
        },
        "capacity": 15
    },
    {
        "id": "EDI004",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 5
    },
    {
        "id": "EDI005",
        "role": "student",
        "categories": {
            "Cognitive",
            "Social"
        },
        "capacity": 8
    },
        {
        "id": "EDI006",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 5
    }
]

In [9]:
import operator

def match(applicants, editors):
    
    matches = []
    
    for applicant in applicants:
        
        potential_editors = asfpy.editors_by_categories(editors, applicant["categories"])
        
        if asfpy.capacity(potential_editors) == 0:
        # If the editing capacity for an applicant is 0, continue to next applicant    
            continue
        else:
            
            faculty_editors = asfpy.editors_by_role(potential_editors, "faculty")
            student_editors = asfpy.editors_by_role(potential_editors, "student")
            
            _match = {
                "applicant": applicant["id"]
            }
        
            if asfpy.capacity(faculty_editors) > 0:
            # If at least one faculty member is available for an applicant,
            # find the best possible match and assign.
                
                hc = asfpy.find_highest_capacity_editors(applicant, faculty_editors)
                
                faculty_in_category = sorted(
                    asfpy.editors_by_categories(faculty_editors, {hc}), 
                    key = operator.itemgetter("capacity"),
                    reverse = True
                )
                
                faculty_id = faculty_in_category[0]["id"]
                
                _match["faculty"] = faculty_id
                
                for e in editors:
                    if e["id"] == faculty_id:
                        e["capacity"] -= 1
                        
            elif applicant["is_flexible"]:
            # If applicant is flexible on two student editors' feedback when no faculty are available,
            # match a student in place of faculty editor, possible.
                pass
            
            else:
            # When no faculty editors are available and the applicant is not 'flexible' on
            # two student editors instead, simply continue to next applicant.
                continue
            
            if asfpy.capacity(student_editors) > 0:               
                
                # _match["student"] = student_id
                # ...do similar to above
                pass
            
            # Append the match to list matches
            
            matches.append(_match)
            
    return matches

In [8]:
from copy import deepcopy
_editors = deepcopy(editors)

p_applicants = asfpy.prioritize(applicants, rank_method = asfpy.asfp_rank)

match(p_applicants, _editors)

[{'id': 'EDI001', 'role': 'faculty', 'categories': {'Clinical', 'Developmental'}, 'capacity': 3}]
[{'id': 'EDI003', 'role': 'faculty', 'categories': {'Cognitive', 'Affective'}, 'capacity': 15}]
[{'id': 'EDI002', 'role': 'faculty', 'categories': {'Affective'}, 'capacity': 2}, {'id': 'EDI003', 'role': 'faculty', 'categories': {'Cognitive', 'Affective'}, 'capacity': 14}]
[{'id': 'EDI001', 'role': 'faculty', 'categories': {'Clinical', 'Developmental'}, 'capacity': 2}]
[{'id': 'EDI001', 'role': 'faculty', 'categories': {'Clinical', 'Developmental'}, 'capacity': 1}]


[{'applicant': 'APP004', 'faculty': 'EDI001'},
 {'applicant': 'APP002', 'faculty': 'EDI003'},
 {'applicant': 'APP003', 'faculty': 'EDI003'},
 {'applicant': 'APP005', 'faculty': 'EDI001'},
 {'applicant': 'APP001', 'faculty': 'EDI001'}]